In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from emnist import get_emnist_data_loaders
train_loader, val_loader, test_loader = get_emnist_data_loaders()

In [3]:
# from models.crnn.crnn import CRNN
# from models.lstm_cnn.lstm_cnn import CNN_LSTM
# import torch.nn as nn
# from training.trainer import Trainer
# import hyperparameter as hp
# import torch
# from evaluation.loss import plot_loss
# from torch.optim.lr_scheduler import StepLR, MultiStepLR, ExponentialLR, CosineAnnealingLR, LambdaLR, CyclicLR


# crnn = CRNN(hp.GRU_HIDDEN_SIZE, hp.GRU_NUM_LAYERS, hp.NUM_CLASSES, hp.CNN_OUTPUT_HEIGHT)
# lstm = CNN_LSTM(hp.GRU_HIDDEN_SIZE, hp.GRU_NUM_LAYERS, hp.NUM_CLASSES, hp.CNN_OUTPUT_HEIGHT)

# models = {
#     "crnn": crnn,
#     "lstm_cnn": lstm
# }

# crnn_criterion = nn.CTCLoss(blank=hp.BLANK_LABEL, reduction='mean', zero_infinity=True)
# lstm_criterion = nn.CTCLoss(blank=hp.BLANK_LABEL, reduction='mean', zero_infinity=True)

# criterions = {
#     "crnn": crnn_criterion,
#     "lstm_cnn": lstm_criterion
# }

# optimizers = {
#     "crnn_adam": torch.optim.Adam(crnn.parameters(), lr=hp.LEARNING_RATE),
#     "crnn_adam_w": torch.optim.AdamW(crnn.parameters(), lr=hp.LEARNING_RATE),
#     "crnn_adam_w_regulise": torch.optim.AdamW(crnn.parameters(), lr=hp.LEARNING_RATE),
#     "lstm_cnn_adam": torch.optim.Adam(crnn.parameters(), lr=hp.LEARNING_RATE),
#     "lstm_cnn_adam_w": torch.optim.AdamW(crnn.parameters(), lr=hp.LEARNING_RATE),
#     "lstm_cnn_adam_w_regulise": torch.optim.AdamW(crnn.parameters(), lr=hp.LEARNING_RATE),
# }

# schedulers = {
#     "crnn_adam": StepLR(optimizers["crnn_adam"], step_size=5, gamma=0.5),
#     "crnn_adam_w": MultiStepLR(optimizers["crnn_adam_w"], milestones=[5, 10, 15], gamma=0.5),
#     "crnn_adam_w_regulise": ExponentialLR(optimizers["crnn_adam_w_regulise"], gamma=0.95),
#     "lstm_cnn_adam": CosineAnnealingLR(optimizers["lstm_cnn_adam"], T_max=10),
#     "lstm_cnn_adam_w": LambdaLR(optimizers["lstm_cnn_adam_w"], lr_lambda=lambda epoch: 0.95 ** epoch),
#     "lstm_cnn_adam_w_regulise": CyclicLR(optimizers["lstm_cnn_adam_w_regulise"], base_lr=0.001, max_lr=0.1)
# }


# crnn_trainer = Trainer(models["crnn"], criterions["crnn"], optimizers["crnn"], train_loader, val_loader,test_loader, epochs=hp.EPOCHS, args={
#         "blank_label": hp.BLANK_LABEL,
#         "cnn_output_width": hp.CNN_OUTPUT_WIDTH,
#     })
# lstm_trainer =  Trainer(models["lstm_cnn"], criterions["lstm_cnn"], optimizers["lstm_cnn"], train_loader, val_loader, test_loader, epochs=hp.EPOCHS, args={
#         "blank_label": hp.BLANK_LABEL,
#         "cnn_output_width": hp.CNN_OUTPUT_WIDTH,
#     })

# trainers = {
#     "crnn": crnn_trainer,
#     "lstm_cnn": lstm_trainer
# }


In [4]:
from models.crnn.crnn import CRNN
from models.lstm_cnn.lstm_cnn import CNN_LSTM
import torch.nn as nn
from training.trainer import Trainer
import hyperparameter as hp
import torch
from torch.optim.lr_scheduler import StepLR, MultiStepLR, ExponentialLR, CosineAnnealingLR, LambdaLR, CyclicLR

# Define your models
models = {
    "crnn": CRNN(hp.GRU_HIDDEN_SIZE, hp.GRU_NUM_LAYERS, hp.NUM_CLASSES, hp.CNN_OUTPUT_HEIGHT),
    "lstm_cnn": CNN_LSTM(hp.GRU_HIDDEN_SIZE, hp.GRU_NUM_LAYERS, hp.NUM_CLASSES, hp.CNN_OUTPUT_HEIGHT)
}

# Define your criteria
criteria = {
    "crnn": nn.CTCLoss(blank=hp.BLANK_LABEL, reduction='mean', zero_infinity=True),
    "lstm_cnn": nn.CTCLoss(blank=hp.BLANK_LABEL, reduction='mean', zero_infinity=True)
}

# Create dictionaries for optimizers and schedulers
optimizer_configs = {
    "adam": torch.optim.Adam,
    "adam_w": torch.optim.AdamW,
    "adam_w_regulise": torch.optim.AdamW
}

scheduler_configs = {
    "step_lr": (StepLR, {"step_size": 5, "gamma": 0.5}),
    "multi_step_lr": (MultiStepLR, {"milestones": [5, 10, 15], "gamma": 0.5}),
    "exponential_lr": (ExponentialLR, {"gamma": 0.95}),
    "cosine_annealing_lr": (CosineAnnealingLR, {"T_max": 10}),
    "lambda_lr": (LambdaLR, {"lr_lambda": lambda epoch: 0.95 ** epoch}),
    # "cyclic_lr": (CyclicLR, {"base_lr": 0.001, "max_lr": 0.1})
}

# Initialize an empty dictionary to store trainers
trainers = {}

# Iterate through models, criteria, optimizers, and schedulers to create trainers
for model_name, model in models.items():
    for optimizer_name, optimizer_class in optimizer_configs.items():
        if optimizer_name == "adam_w_regulise":
            optimizer = optimizer_class(model.parameters(), lr=hp.LEARNING_RATE, weight_decay=hp.WEIGHT_DECAY)
        else:
            optimizer = optimizer_class(model.parameters(), lr=hp.LEARNING_RATE)
        for scheduler_name, (scheduler_class, scheduler_args) in scheduler_configs.items():
            scheduler = scheduler_class(optimizer, **scheduler_args)
            criterion = criteria[model_name]
            trainer_name = f"{model_name}_{optimizer_name}_{scheduler_name}_trainer"
            trainer = Trainer(
                model, model_name, criterion, optimizer, train_loader, val_loader, test_loader,
                epochs=hp.EPOCHS, args={"blank_label": hp.BLANK_LABEL, "cnn_output_width": hp.CNN_OUTPUT_WIDTH}
            )
            trainers[trainer_name] = trainer
print(f"Prepared {len(trainers)} models for training.")


Prepared 30 models for training.


In [ ]:
metrics_global = {}
for model_name, trainer in trainers.items():
    print(f"Training {model_name}")
    metrics = trainer.train_validate_test()
    metrics_global[model_name] = metrics
    print(f"Finished training {model_name}")

    # Save the trained model
    torch.save(trainer.model.state_dict(), 'models/' + model_name + '.pth')

Training crnn_adam_step_lr_trainer
100%|██████████| 110/110 [00:31<00:00,  3.54it/s]
EPOCH 1/10 - TRAINING. Correct: 1/7000 = 0.0001 - Average CER Score: 0.962
100%|██████████| 2999/2999 [00:12<00:00, 241.11it/s]
[0.5, 1.0, 0.0, 0.5, 0.75, 0.0, 0.6666666865348816, 1.0, 0.3333333432674408, 1.0, 0.0, 1.0, 0.75, 1.0, 0.5, 1.0, 0.75, 0.6666666865348816, 1.0, 0.0, 1.0, 0.5, 1.0, 0.5, 0.5, 1.0, 1.0, 1.0, 0.5, 0.6666666865348816, 0.5, 0.6666666865348816, 1.0, 0.0, 0.6666666865348816, 1.0, 1.0, 1.0, 0.5, 1.0, 0.0, 1.0, 0.0, 1.0, 0.25, 1.0, 0.6666666865348816, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.25, 1.0, 0.5, 0.5, 0.6666666865348816, 0.6666666865348816, 0.0, 0.6666666865348816, 0.3333333432674408, 0.25, 0.25, 0.6666666865348816, 0.0, 1.0, 0.5, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.5, 0.0, 0.0, 1.0, 0.5, 0.3333333432674408, 0.0, 0.20000000298023224, 0.0, 1.0, 0.5, 0.0, 1.0, 1.0, 0.75, 0.0, 0.5, 1.0, 1.0, 0.5, 1.0, 0.0, 1.0, 0.5, 1.0, 1.0, 0.6666666865348816, 0.6666666865348816, 1.

: 

: 

In [5]:
from evaluation.loss import plot_loss
for model_name, trainer in trainers.items():
    # Load model

    base_path = 'models/'
    model_path = model_name + '.pth'
    model = trainer.model.to("cpu")
    trainer.model.load_state_dict(torch.load(model_path))

    # Plot the trained model
    plot_loss(model_name=model_name, train_loss_per_epoch=trainer.epoch_train_cers, val_loss_per_epoch=trainer.epoch_val_cers, figure_evaluation_dir=base_path)

FileNotFoundError: [Errno 2] No such file or directory: 'crnn_adam_step_lr_trainer.pth'

In [ ]:
for model_name, trainer in trainers.items():
    trainer.load_model(f'models/{model_name}/model.pth')
    trainer.test(plot_n=1)


100%|██████████| 1/1 [00:00<00:00, 184.91it/s]


AttributeError: 'Trainer' object has no attribute 'epoch_test_cers'